In [ ]:
import pandas as pd
import copy 
import time

In [ ]:
def ReadData(filepath):
    df = pd.read_csv(filepath)
    df[['response_tweet_id', 'in_response_to_tweet_id']] = df[['response_tweet_id','in_response_to_tweet_id']].fillna(value=0)
    print(df.shape)
    print(df.dtypes)
    df.head()
    return df

In [ ]:
#making apple data
def MakeAndReturnCompanyData(df,CompanyName):
    df_company = df.loc[df['author_id'] == CompanyName]
    print(df_company.shape)
    print("not ",CompanyName ," data is",round(df_company.shape[0]/df.shape[0]*100,4) , "% of the total data")
    df_company.head()
    return df_company

In [ ]:
#making non apple data
def MakeAndReturnNonCompanyData(df,CompanyName):
    df_company = df.loc[df['author_id'] != CompanyName]
    print(df_company.shape)
    print("not ",CompanyName ," data is",round(df_company.shape[0]/df.shape[0]*100,4) , "% of the total data")
    df_company.head()
    return df_company

### Functions To Make Threads of Apple Support

In [ ]:
def getStringListFromCommaSeperatedString(my_string):

    if "," in str(my_string):
        return [x.strip() for x in my_string.split(',')]
    else:
        return [my_string]

In [ ]:
def IterateResponseTweetIDs(response_tweet_ids,df,thread_id):
    for response_tweet_id in response_tweet_ids:
        df = processThread(df,response_tweet_id,thread_id)
    return df

In [ ]:
#returns rows which has the value:columnValue in column : columnName
def GetSpecificColumnValueRows(df,columnName,columnValue):
    return df.loc[df[columnName] == columnValue ]

In [ ]:
##start of the thread pe khara hun, aage iterate karna hai ab.


def processThread(df,tweet_id,thread_id):
    print("tweet_id =",tweet_id,end="  -->")
    if ((df.tweet_id==int(tweet_id)).any()):

        df.loc[df['tweet_id'] == int(tweet_id), 'checked'] = 0
        df.loc[df['tweet_id'] == int(tweet_id), 'thread_id'] = thread_id
        
        response_tweet_ids = getStringListFromCommaSeperatedString(GetSpecificColumnValueRows(df,"tweet_id",int(tweet_id))["response_tweet_id"].values[0])

        if( 0 not in response_tweet_ids):
            print("child responses : ",response_tweet_ids ,end="  -->")
            IterateResponseTweetIDs(response_tweet_ids,df,thread_id)
        else:                
            print("Thread end",end="  -->")
    else:
        print("not found",end="  -->")
    return df


In [ ]:
def isCompanyOneOfTheResponseTweets(response_tweet_ids,df_company):
    for response_tweet_id in response_tweet_ids:
        if int(response_tweet_id) in df_company["tweet_id"].tolist():
            return True

    return False

In [ ]:
def ProcessDataFrame(df,df_company,startingThreadNumber,CompanyName):
    thread_id = startingThreadNumber
    df["checked"]=1   #0 for true i.e its checked and 1 for false i.e its not checked
    df["thread_id"]=0
    start = time.clock()
    end2=time.clock()
    for row in df.itertuples():
        index = row.Index+2

        response_tweet_ids = getStringListFromCommaSeperatedString(str(row.response_tweet_id))

        #index>329093

        if row.checked==1 and row.thread_id==0 and row.author_id != CompanyName and row.inbound == True and isCompanyOneOfTheResponseTweets(response_tweet_ids,df_company)==True and row.in_response_to_tweet_id==0:        
            tweet_id = row.tweet_id

            print(" Thread ID = ,", thread_id,"\n-------------------------")
            df= processThread(df,tweet_id,thread_id)
            thread_id+=1 
            print("")     

        if(index%50000==0):
            print("\n****************************************************************************************")
            print("Time taken to process 50k threads: ",time.clock()-end2 ," seconds")
            print("total tweets processed = ",index,end=" ,")
            print("****************************************************************************************\n")
            end2=time.clock()

    end = time.clock()
    print("Total time taken for complete processing : ",end-start," seconds")
    return df
    

###  Functions to Make data in the format for support of chatbot training

In [ ]:
def MakeDataForChatbotTraining(df,author_id):
    print("\n Start Processing data.")

    first_tweet = df[df.in_response_to_tweet_id==0 & df.inbound ]
    tweets_and_replies = pd.merge(first_tweet, df, left_on='tweet_id', 
                                      right_on='in_response_to_tweet_id')
    tweets_and_replies = tweets_and_replies[tweets_and_replies.inbound_y ^ True]

    df_to_company_support = tweets_and_replies[tweets_and_replies.author_id_y==author_id]
    
    print(df_to_company_support.shape)
    print("\n End Processing data.")

    return df_to_company_support

In [ ]:
#for writing to file
def WriteToFile(df_to_company_support,filepath):
    f= open(filepath,"w", encoding='utf-8')
    print("\n Start Writing.")

    for row in df_to_company_support.itertuples():
        index = row.Index
        text_x = (row.text_x).encode('ascii', 'ignore')
        text_y = (row.text_y).encode('ascii', 'ignore')

        text_x = text_x.replace("\n".encode('ascii', 'ignore'),"".encode('ascii', 'ignore'))
        text_x = text_x.replace("\r".encode('ascii', 'ignore'),"".encode('ascii', 'ignore'))
        text_y = text_y.replace("\n".encode('ascii', 'ignore'),"".encode('ascii', 'ignore'))
        text_y = text_y.replace("\r".encode('ascii', 'ignore'),"".encode('ascii', 'ignore'))

        value = text_x + "\t".encode('ascii', 'ignore') + text_y + "\n".encode('ascii', 'ignore')

        value = (value).decode("utf-8")
        f.write(value)
    print("\n Done Writing.")
    f.close()



In [ ]:

#for reading from file
def ReadBinaryFile(filepath,LinesToPrint):
    fread = open(filepath,"rb") 
    print("\n Start Rading. \n")

    lines = fread.readlines()
    numberOfTweetAndResponsesToPrint = LinesToPrint
    for index,line in enumerate(lines):
        print(line)
        if (index+1) ==numberOfTweetAndResponsesToPrint:
            break
    print("\n Done Reading")
    fread.close()

## Calling Functions to make company Specific threads

In [ ]:
# Run this cell if you want to make complete threads of apple support

print("Reading Data")
df = ReadData("./twcs/twcs.csv")

print("Making Company data")
df_company = MakeAndReturnCompanyData(df,"AppleSupport")

print("Making Non Company data")
df_notCompany = MakeAndReturnNonCompanyData(df,"AppleSupport")

print("Processing Data")
df = ProcessDataFrame(df,df_company,1,"AppleSupport")

## Calling Functions to Format Data For Chatbot Training

In [ ]:
df_to_company_support = MakeDataForChatbotTraining(df,"AppleSupport")

WriteToFile(df_to_company_support,"./Chatbot Training Data/formatted_AppleSupport_tweets.txt")

ReadBinaryFile("./Final_Final_Data/formatted_AppleSupport_tweets.txt",1)
